<a href="https://colab.research.google.com/github/rizzken/PreAppointmentPatientSummary-Eval/blob/main/01_data_quality_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dirty_patients.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# 1. Basic dataset information
print("Dataset shape (rows, columns):", df.shape)
print("\nData types:\n", df.dtypes)

missing_per_col = df.isnull().sum()
missing_cols = missing_per_col[missing_per_col > 0]
if not missing_cols.empty:
  print("\nMissing values per column:\n", missing_cols)

Dataset shape (rows, columns): (115, 28)

Data types:
 id                      object
birthdate               object
deathdate               object
ssn                     object
drivers                float64
passport                object
prefix                  object
first                   object
last                    object
suffix                  object
maiden                  object
marital                 object
race                    object
ethnicity               object
gender                  object
birthplace              object
address                 object
city                    object
state                   object
county                  object
fips                   float64
zip                      int64
lat                    float64
lon                    float64
healthcare_expenses    float64
healthcare_coverage    float64
income                 float64
location                object
dtype: object

Missing values per column:
 deathdate              100
drivers

In [31]:
# 2. Check for duplicate rows
print("Number of fully duplicate rows:", df.duplicated().sum())

# If Patient ID, passport, drivers have a unique values)
if 'id' in df.columns:
    print("Duplicated Patient IDs:", df.duplicated(subset=['id']).sum())
if 'id' in df.columns:
    print("Duplicated Patient Drivers", df.duplicated(subset=['drivers']).sum())
if 'id' in df.columns:
    print("Duplicated Patient Passports:", df.duplicated(subset=['passport']).sum())

Number of fully duplicate rows: 2
Duplicated Patient IDs: 5
Duplicated Patient Drivers 24
Duplicated Patient Passports: 34


In [32]:
# 3. Numerical columns validation
numerical_cols = ['lat', 'lon', 'healthcare_expenses', 'healthcare_coverage', 'income']
money_cols =['healthcare_expenses', 'healthcare_coverage', 'income']

for col in numerical_cols:
    if col in df.columns:
        print(f"\n{col} statistics:\n", df[col].describe())
for col in money_cols:
    if col in df.columns:
      negative_rows = df[df[col] < 0]
      if not negative_rows.empty:
        print(f"\nNegative values in '{col}' column:", df[df[col] < 0].index.tolist())


lat statistics:
 count    115.000000
mean      32.968196
std        1.303361
min       30.254587
25%       32.289273
50%       33.308101
75%       33.685242
max       34.957788
Name: lat, dtype: float64

lon statistics:
 count    115.000000
mean     -86.595327
std        0.762670
min      -88.297123
25%      -86.929071
50%      -86.673893
75%      -85.965817
max      -85.095567
Name: lon, dtype: float64

healthcare_expenses statistics:
 count    9.400000e+01
mean     4.732806e+05
std      1.498127e+06
min     -5.000000e+02
25%      2.272378e+04
50%      5.820105e+04
75%      1.478484e+05
max      9.999999e+06
Name: healthcare_expenses, dtype: float64

healthcare_coverage statistics:
 count    9.200000e+01
mean     5.123183e+05
std      9.847146e+05
min      0.000000e+00
25%      1.304884e+04
50%      7.273544e+04
75%      6.078895e+05
max      5.791878e+06
Name: healthcare_coverage, dtype: float64

income statistics:
 count        86.000000
mean      65169.823331
std       96107.06794

In [33]:
# 4. Categorical columns inspection
categorical_cols = ['id', 'drivers', 'passport', 'first', 'last', 'address']

for col in categorical_cols:
   if col in df.columns:
      print(f"\nMissing values in '{col}' column:", df[col].isna().sum())
      if df[col].isna().sum() != 0:
         print(f"\nIn rows: ",  df[df[col].isna()].index.tolist())


Missing values in 'id' column: 0

Missing values in 'drivers' column: 20

In rows:  [0, 3, 12, 14, 17, 20, 37, 44, 45, 46, 50, 53, 56, 59, 67, 69, 85, 89, 99, 101]

Missing values in 'passport' column: 30

In rows:  [0, 2, 3, 8, 12, 14, 17, 20, 21, 22, 32, 35, 37, 41, 44, 45, 46, 50, 53, 56, 59, 67, 69, 76, 85, 89, 99, 100, 101, 102]

Missing values in 'first' column: 0

Missing values in 'last' column: 0

Missing values in 'address' column: 0


In [34]:
#4 Check Gender consistency
print(df['gender'].value_counts())
valid_gender = ['male', 'female']
gender_typo = df[df['gender'].notna() & ~df['gender'].str.strip().str.lower().isin(valid_gender)]
print(f"\nTypo in gender in rows:", gender_typo.index.tolist())

gender
female    62
male      50
mle        1
fale       1
ale        1
Name: count, dtype: int64

Typo in gender in rows: [9, 59, 102]


In [35]:
#5 Check Birth and Death dates logic.
df['birthdate'] = pd.to_datetime(df['birthdate'], errors='coerce')
df['deathdate'] = pd.to_datetime(df['deathdate'], errors='coerce')

empty_birth = df[df['birthdate'].isna()]
birth_before = df[df['birthdate'] < '1900-01-01']
birth_future = df[df['birthdate'] > pd.to_datetime("today")]
death_before_birth = df[df['deathdate'].notna() & (df['deathdate'] < df['birthdate'])]
death_future = df[df['deathdate'] > pd.to_datetime("today")]

print(f"\nRows with empty birthdate:", empty_birth.index.tolist())
print(f"\nRows with birthdate before 1900:" , birth_before.index.tolist())
print(f"\nRows with birthdate in the future:", birth_future.index.  tolist())
print(f"\nRows with deathdate before birthdate:", death_before_birth.index.tolist())
print(f"\nRows with deathdate in the future:", death_future.index.tolist())


# Age calculation for alive patients (deathdate.isna())
today = pd.to_datetime("today").normalize()
df['age'] = pd.NA

alive_mask = df['deathdate'].isna()
df.loc[alive_mask, 'age'] = (today - df.loc[alive_mask, 'birthdate']).dt.days // 365



Rows with empty birthdate: []

Rows with birthdate before 1900: []

Rows with birthdate in the future: [57]

Rows with deathdate before birthdate: [29, 33, 43, 75]

Rows with deathdate in the future: []
